In [ ]:
# Replace part of the OUTPUT_PATH to create a new folder 
# for the detection results

OUTPUT_PATH = "Temp/LoanApplications_Offline/"
NEW_OUTPUT_PATH = "Temp/LoanApplications_Offline__DETECTION/"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

# My packages
from source import parse_mxml as pm
from source import log_representation as lr
from source import plots as plts
from source import drift_detection as dd
from source import drift_localization as dl
from source import offline_streaming_clustering as off_sc

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import hdbscan
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed, parallel_backend

import random
random.seed(42)

import os
import glob

import gc
gc.enable()

from scipy.spatial import distance
from sklearn.base import clone as sk_clone 

from copy import deepcopy

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)

In [ ]:
def insensitive_glob(pattern):
    def either(c):
        return '[%s%s]' % (c.lower(), c.upper()) if c.isalpha() else c
    return glob.glob(''.join(map(either, pattern)))

def if_any(string, lista):
    # If the string contains any of the values
    # from the list 'lista'
    for l in lista:
        if l in string:
            return True
    return False

In [ ]:
# List log files
logs = insensitive_glob(r"../../../../../../../Datasets/Business_Process_Drift_Logs/Logs/*/*k.MXML")
logs = [x.replace('\\', '/') for x in logs if "2.5" not in x]
# logs = [x for x in logs if "2.5" not in x]

In [ ]:
# reference objects and map them to strings in dict 
# used in further methods
objects = {
    "model": {
#         "kmeans__k=6": KMeans(n_clusters=6, random_state=42),
#         "kmeans__k=3": KMeans(n_clusters=3, random_state=42),
#         "kmeans__k=2": KMeans(n_clusters=2, random_state=42),
#         "DBSCAN__eps=05ms=5": DBSCAN(eps=0.5, min_samples=5, metric='euclidean'),
#         "DBSCAN__eps=1ms=4": DBSCAN(eps=1, min_samples=4, metric='euclidean'),
#         "DBSCAN__eps=2ms=3": DBSCAN(eps=2, min_samples=3, metric='euclidean'),
        "HDBSCAN__noparams": hdbscan.HDBSCAN(gen_min_span_tree=True, allow_single_cluster=True)
    },
    
    "representation": {
        "activity_binary": lr.get_binary_representation,
        "activity_frequency": lr.get_frequency_representation,
        
        "transitions_binary": lr.get_binary_transitions_representation,
        "transitions_frequency": lr.get_frequency_transitions_representation,
        
        "activity_tfidf": lr.get_tfidf_representation,
        "transitions_tfidf": lr.get_tfidf_transitions_representation,
        
        "activity_transitions_frequency": lr.get_activity_transitions_frequency_representation,
        "activity_transitions_binary": lr.get_activity_transitions_binary_representation
    }
#     "representation": {
#         "activity_binary": lambda x: lr.get_binary_representation(lr.get_traces_as_tokens(x)),
#         "activity_frequency": lambda x: lr.get_frequency_representation(lr.get_traces_as_tokens(x)),
#         "transitions_binary": lambda x: lr.get_binary_transitions_representation(lr.get_traces_as_tokens(x)),
#         "transitions_frequency": lambda x: lr.get_frequency_transitions_representation(lr.get_traces_as_tokens(x)),
#         "activity_transitions_frequency": lambda x: pd.concat([lr.get_frequency_transitions_representation(lr.get_traces_as_tokens(x)), lr.get_frequency_representation(lr.get_traces_as_tokens(x))],axis=1),
#         "activity_transitions_binary": lambda x: pd.concat([lr.get_binary_transitions_representation(lr.get_traces_as_tokens(x)), lr.get_binary_representation(lr.get_traces_as_tokens(x))],axis=1)
#     }
}

In [ ]:
# change patterns and they supported representations
activity_binary_drifts = ["cb", "cf", "cm", "fr", "pm", "re", "rp"]
activity_frequency_drifts = activity_binary_drifts + ["cp", "lp"]

transitions_binary_drifts = activity_frequency_drifts + ["cd", "pl", "sw"]
transitions_frequency_drifts = transitions_binary_drifts

activity_tfidf_drifts = transitions_binary_drifts
transitions_tfidf_drifts = transitions_binary_drifts

activity_transitions_frequency_drifts = transitions_binary_drifts
activity_transitions_binary_drifts = transitions_binary_drifts

### Pipeline Offline Clustering

In [ ]:
def read_file_and_run_clustering_pipeline(args, return_result=False):
    """
    Read an event log file, represent it into a feature vector space and
    run the trace clustering method over windows. This method outputs results
    as gzip csv files into the "OUTPUT_PATH" folder, or return the result 
    as DataFrame when return_result = True.
    
    Parameters:
    -----------
        args (dict): Dictionary with the parameters and the log_file path
            requiring the following keys:
                example = {
                    'log': <PATH TO LOG_FILE>,
                    'representation': <KEY TO REPRESENTATIONS IN 'objects'>,
                    'parameters': [{
                        'model': <KEY TO MODEL IN 'objects'>, 
                        'sliding_window': <WHETHER TO USE SLIDING WINDOW>,
                        'window_size': <SIZE OF TRACE WINDOW TO USE>,
                        'sliding_step': <STEP OF SLIDING WINDOW>
                    }
        return_result (bool): Whether to return the result as DataFrame
            
    """
    
    # Treat file name to structure size and log type
    split = args["log"].split("/")
    
    # Parse change pattern name
    cd_name = split[-2]
    log_name = split[-1][:-5]

    # Parse size of the event_log
    log_size = log_name.replace(cd_name, "").replace("new_", "")
    log_size = int(float(log_size.replace("k", "")) * 1000)
    
    # Set up true drifts indexes
#     y_true = [x for x in range(int(log_size/10), log_size, int(log_size/10))]
    
    try:
        # Read log and apply trace representation technique
        log_read = pm.all_prep(open(args["log"]))
        tokens = lr.get_traces_as_tokens(log_read)
#         df = objects["representation"][args["representation"]](log_read)
        
        for p in args["parameters"]:
            # String to identify results when exporting files
            tipo_mudanca = cd_name.replace("new_", "")
            
            cached_info = "_".join([
                tipo_mudanca,
                str(log_size),
                p["model"],
                args["representation"],
                str(p["window_size"]),
                str(p["sliding_window"])
            ])
            
#             print(cached_info)
            
            # If already exists, return if needed
            file_to_export = OUTPUT_PATH + tipo_mudanca + '/' + cached_info + '.pickle.gzip'
            if os.path.exists(file_to_export):
                if return_result:
                    r_ = pd.read_pickle(
                        file_to_export,
                        compression='gzip'
                    )
                    return r_
                else:
                    continue
            
            # If file does not exists, run trace clustering step and export file
            all_metrics = off_sc.run_offline_clustering_window(
                tokens,
                objects["representation"][args["representation"]],
                sk_clone(objects["model"][p["model"]]),
                p["window_size"],
#                 df,
                p["sliding_window"],
                sliding_step=p['sliding_step']
            )
            
            # Set up true drifts indexes and append
            y_true = list(range(int(len(tokens)/10), len(tokens), int(len(tokens)/10)))
            all_metrics["y_true"] = all_metrics.apply(lambda x: y_true, axis = 1)
            
            if return_result:
                return all_metrics
            else:
                try:
                    os.makedirs(OUTPUT_PATH + tipo_mudanca + '/')
                except:
                    pass
                all_metrics.to_pickle(file_to_export, compression="gzip")

            gc.collect()
    except Exception as e:
        raise e

#### Run pipeline for specific case(s)

In [ ]:
# read_file_and_run_clustering_pipeline({
#     'log': logs[0],
#     'representation': 'activity_binary',
#     'parameters': [{
#         'model': 'HDBSCAN__noparams', 
#         'sliding_window': False,
#         'window_size': 75,
#         'sliding_step': 1
#     }]
# }, return_result=True)

### Run Experiments with several parameters combinations

In [ ]:
# Trace clustering parameters
grid_parameters = list(ParameterGrid({
    "sliding_window": [False],    
    "window_size": [100, 125, 150, 200],
    'sliding_step': [1],
    "model": [
#         'kmeans__k=6',
#         'kmeans__k=3',
#         'kmeans__k=2',
#         "DBSCAN__eps=05ms=5",
#         "DBSCAN__eps=1ms=4",
#         "DBSCAN__eps=2ms=3",
        "HDBSCAN__noparams"
    ] 
}))

# Trace vector representations
grid_logs = list(ParameterGrid([
    { "log": [x for x in logs if if_any(x, activity_binary_drifts)],
        "representation": ["activity_binary"]},
    {"log": [x for x in logs if if_any(x, activity_frequency_drifts)],
        "representation": ["activity_frequency"]},
    
    { "log": [x for x in logs if if_any(x, transitions_binary_drifts)],
        "representation": ["transitions_binary"]},
    
    { "log": [x for x in logs if if_any(x, transitions_frequency_drifts)],
        "representation": ["transitions_frequency"]},
    
    { "log": [x for x in logs if if_any(x, activity_tfidf_drifts)],
        "representation": ["activity_tfidf"]},
    {"log": [x for x in logs if if_any(x, transitions_tfidf_drifts)],
        "representation": ["activity_transitions_binary"]},
    
    {"log": [x for x in logs if if_any(x, activity_transitions_frequency_drifts)],
        "representation": ["activity_transitions_frequency"]},
    {"log": [x for x in logs if if_any(x, activity_transitions_binary_drifts)],
        "representation": ["activity_transitions_binary"]}
]))

# Combining all parameters
combs = []
for x in grid_logs:
    dic = x.copy()
    dic['parameters'] = grid_parameters
    
    combs.append(dic)

len(combs), len(grid_parameters), len(combs) * len(grid_parameters) 

#### Run parallely

In [ ]:
final_resp = Parallel(n_jobs=-1)(
    delayed(read_file_and_run_clustering_pipeline)(comb) for comb in tqdm_notebook(combs)
)
gc.collect()

### Detection Pipeline

In [ ]:
# Drift detection parameters
drift_config = list(ParameterGrid([
    {
        "rolling_window": [3, 4]#[3, 4, 5]
        ,"std_tolerance": [3]#[1.25, 1.5, 1.75, 2]
        ,'min_tol': [0.025]#[0.01, 0.007, 0.005, 0.003] 
    }
]))
print(len(drift_config))

In [ ]:
# List all files obtained after the trace clustering pipeline

clusterizacoes = glob.glob(OUTPUT_PATH + "*/*.pickle.gzip")
print(len(clusterizacoes))

In [ ]:
# Combinations to run

combs_deteccao = []
for log in clusterizacoes:
    combs_deteccao.append({
        'input': log,
        'combinations': drift_config
    })
print(len(combs_deteccao))

In [ ]:
def drift_detect_pipeline(args, return_results=False):
    """
        Runs the drift detection method based on the output from the trace
        clustering pipeline for different combination of parameters and every
        feature from tracking the trace clustering evolution. The
        outputs are into a new folder named by the NEW_OUTPUT_PATH variable in 
        gzip csv files.
        
        Parameters:
        -----------
            args (dict): Dictionary with the parameters and the log_file path
                requiring the following keys:
                    example = {
                    }
    """
    # Read file
    all_metrics = pd.read_pickle(args["input"], compression='gzip')
    
    # Parse information from file name
    path_file = args["input"].replace(".pickle.gzip", "").split('\\')
    args.update({
        "tipo_mudanca": path_file[-1].split('_')[0],
        "log_size": int(path_file[-1].split('_')[1]),
        "model": "_".join(path_file[-1].split('_')[2:5]),
        "representation": "_".join(path_file[-1].split('_')[5:-2]),
        "window_size": path_file[-1].split('_')[-2],
        "sliding_window": path_file[-1].split('_')[-1]
    })
    
    # Run detection for every combination of parameter    
    for combination in args['combinations']:
        c = deepcopy(combination)
        c.update({
            'input': args['input'],
            'tipo_mudanca': args['tipo_mudanca'],
            'log_size': args['log_size'],
            'model': args['model'],
            'representation': args['representation'],
            'window_size': args['window_size'],
            'sliding_window': args['sliding_window']
        })
        
        if return_results:
            return __drift_detect_pipeline(
                all_metrics, c, return_results
            )
        else:
            __drift_detect_pipeline(
                all_metrics, c, return_results
            )
    

def __drift_detect_pipeline(all_metrics, args, return_results=False):     
    base_name = args["input"].replace(".pickle.gzip", "").replace("\\", "/")
    base_name = base_name.replace(OUTPUT_PATH, NEW_OUTPUT_PATH)
    
    # Create string with parameters to identify file
    to_string = [
        str(args["rolling_window"]),
        str(args["std_tolerance"]).replace(".", "-"), 
        str(args["min_tol"]).replace(".", "-")
    ]
    
    try:
        os.makedirs(base_name)
    except:
        pass
    
    final_name = base_name + "/" + "_".join(to_string) + ".pickle.gzip"
#     final_name = base_name.replace("\\", "/") + "_".join(to_string) + ".pickle.gzip"
    
    if os.path.isfile(final_name):
        if return_results:
            return pd.read_pickle(final_name, compression='gzip')
        else:
            print("Already exists")
            return
    
#     y_true = [x for x in range(int(args['log_size']/10), args['log_size'], int(args['log_size']/10))]
    
    # Runs the drift detection for every feature
    results = []
    for col in all_metrics.select_dtypes(include=np.number).columns:
        if (col not in ["i","test_id"]):
#         if (col not in ["k"] and not col.startswith("diff") ) or col in ["diff_centroids"]:
            r = deepcopy(args)
            r["measure"] = col

            detected_drifts, not_drifts, info = dd.detect_concept_drift(
                all_metrics, 
                col,
                args["rolling_window"],
                args["std_tolerance"],
                args["min_tol"]
            )    
    
            # Calculate classification metrics
            metrics_results = dd.get_metrics(
                detected_drifts,
                not_drifts,
                all_metrics["y_true"].iloc[0], #y_true,
                int(args["window_size"])
            )

            r.update(args)
            r.update(metrics_results)

            results.append(r)

            gc.collect()
    
    # Export as file
    pd.DataFrame(results).to_pickle(
        final_name,
        compression="gzip"
    )
    
    if return_results:
        return pd.DataFrame(results)
    
    # print(col, len(results))

In [ ]:
# detection_results = drift_detect_pipeline({
#     'input': '../LoanApplications_Offline\\cb\\cb_10000_kmeans__k=2_binary_100_False.pickle.gzip',
#     'combinations': [{
#        'min_tol': 0.01,
#        'rolling_window': 3,
#        'std_tolerance': 1.25
#     }, {
#        'min_tol': 0.02,
#        'rolling_window': 3,
#        'std_tolerance': 1.25
#     }]
# }, return_results=True)

#### Run parallely

In [ ]:
final_resp = Parallel(n_jobs=-1)(
    delayed(drift_detect_pipeline)(comb_d) for comb_d in tqdm_notebook(combs_deteccao)
)

## Consolidate Results

In [ ]:
models = [
#     'kmeans__k=6',
#     'kmeans__k=3',
#     'kmeans__k=2',
#     "DBSCAN__eps=05_ms=5",
#     "DBSCAN__eps=1_ms=4",
#     "DBSCAN__eps=2_ms=3",
    "HDBSCAN__noparams"
]

# Function to Read results
def consolidate_results(log):
    return pd.read_pickle(log, compression='gzip')

In [ ]:
for model in models:
    # List all files obtained after the trace clustering pipeline
    deteccoes = glob.glob(NEW_OUTPUT_PATH + "/*/*"+model+"*/*.pickle.gzip")

    # Call function to read results and then consolidate
    all_results = pd.DataFrame()
    all_results = all_results.append(Parallel(n_jobs=-1)(
        delayed(consolidate_results)(log) for log in tqdm_notebook(deteccoes)
    ))
    
    # Export as file
    all_results.to_pickle(
        'Temp/all_results_'+model,
        compression="gzip"
    )

## See Results

In [ ]:
# Import as file
all_results = pd.read_pickle('Temp/all_results_'+models[0], compression='gzip')
print(all_results.shape)

# Group results by all params and get the mean F1 and Delay among all scenarios
# In other words, what combination of params got the best results in all scenarios

params = ["min_tol", "rolling_window", "std_tolerance", "window_size", "measure", "representation"]
scenarios = ["tipo_mudanca", "log_size"]
validation_metrics = ["F1","Delay"]#,"Support","Precision","Recall"]

all_results_grouped = all_results.groupby(params)[validation_metrics].agg(['mean','std'])
all_results_grouped.columns = all_results_grouped.columns.map('_'.join)
# all_results_grouped = all_results_grouped[all_results_grouped['F1_count']>=len(activity_binary_drifts)]
all_results_grouped.sort_values(["F1_mean","Delay_mean"], ascending=[False,True], inplace=True)
all_results_grouped.head(50)

In [ ]:
best_results_by_representation = all_results_grouped.reset_index().groupby("representation").first().reset_index().sort_values(["F1_mean","Delay_mean"], ascending=[False,True])
best_results_by_representation

In [ ]:
best_result = pd.merge(left=all_results
            , right=best_results_by_representation
            , how='inner', on=params)
results_table = best_result.pivot_table(values=["F1","Delay"]
                                        , index=["tipo_mudanca"]
                                        , columns=["representation"]
                                        , aggfunc='mean')\
                                    .sort_index(axis='columns',level=[1,0], ascending=[True,False])

results_table.to_excel('Temp/Results/results_table_'+models[0]+'.xlsx', sheet_name=models[0])
results_table

In [ ]:
results_table['F1'].plot(kind='bar', figsize=(20,10), fontsize=20)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
plt.show()

In [ ]:
results_table_logsize = best_result.pivot_table(values=["F1"]#,"Delay"]
                                                , index=["tipo_mudanca"]
                                                , columns=["representation"
                                                , "log_size"]
                                                , aggfunc='mean')\
                                    .sort_index(axis='columns',level=[1,0], ascending=[True,False])

results_table_logsize.to_excel('Temp/Results/results_table_logsize'+models[0]+'.xlsx', sheet_name=models[0])
results_table_logsize

In [ ]:
shsow_result = pd.merge(left=best_result
            , right=best_results_by_representation.head(1)
            , how='inner', on=params)
shsow_result

In [ ]:
for row in range(0,len(shsow_result)):

    shsow_result_log = [x for x in logs if shsow_result.iloc[row]["tipo_mudanca"] in x 
         and str(shsow_result.iloc[row]["log_size"]/1000) in 
                    str(float(x.split("/")[-1][:-5].replace("k", "").replace(x.split("/")[-2],"")))]
    
    log_read = pm.all_prep(open(shsow_result_log[0]))
    tokens = lr.get_traces_as_tokens(log_read)

    run_df = off_sc.run_offline_clustering_window(
        tokens,
        objects["representation"][shsow_result.iloc[row]['representation']],
        objects["model"][shsow_result.iloc[row]['model']],
        int(shsow_result.iloc[row]['window_size']),
        sliding_window=False,
        sliding_step=1
    )

    drifts, not_drifts, info = dd.detect_concept_drift(
        run_df,
        shsow_result.iloc[row]['measure'],
        rolling_window=shsow_result.iloc[row]['rolling_window'],
        std_tolerance=shsow_result.iloc[row]['std_tolerance'],
        min_tol=shsow_result.iloc[row]['min_tol']
    )

    plts.plot_deteccao_drift(
        run_df,
        shsow_result.iloc[row]['measure'],
        shsow_result.iloc[row]['Drifts_Found'],
        shsow_result.iloc[row]['Resp'],
        info['means'],
        info['lowers'],
        info['uppers'],
        save_png=""
    )
    plt.title("Log: " + shsow_result.iloc[row]["tipo_mudanca"] + str(shsow_result.iloc[row]["log_size"]) 
                  + " - Rep: " + shsow_result.iloc[row]["representation"] 
                  + " - Metric: " + shsow_result.iloc[row]["measure"]
                  + " - F1: " + str(round(shsow_result.iloc[row]["F1"],2))
              , size=30)
    plt.show()

In [ ]:
# all_results.to_excel('Resultados_PL.xlsx')